In [3]:
import pandas as pd
import numpy as np
import csv, json
from urllib.parse import urlparse

In [26]:
def getID(url):
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')
    return int(path_segments[3])
getID("https://www.tiktokv.com/share/video/7315561816673750318/")

7315561816673750318

In [36]:
def processCsv(csvd):
    csvdf = csvd
    position = csvdf.columns.get_loc('video_timestamp') + 1
    csvdf.insert(position, 'video_time_watched', np.nan)
    return csvdf

In [58]:
def processJson(jsond): #for Sec2Gr2 files
    jsondf = jsond
    jsondf['video_id'] = jsondf['Link'].apply(getID)
    return jsondf

def processJson2(jsonfilename): #for Sec1Gr1 files
    filename = f'{jsonfilename}'
    with open(filename, 'r') as file:
        jsondata = json.load(file)

    jsondict = {'Date':[],'Link':[],'video_id':[]}
    for video in jsondata['data']:
        jsondict['Link'].append(video[1])
        jsondict['video_id'].append(getID(video[1])) 
        jsondict['Date'].append(video[0])

    jsondf = pd.DataFrame(jsondict)
    return jsondf
    

In [7]:
#for ech pair create new df with date
def merged_date(csvd, jsond):
    csvdf = csvd
    jsondf = jsond
    csvrow = 0
    jsonrow = 0
    for id in jsondf['video_id']: #for each video watched in json
        if id == csvdf['video_id'][csvrow]: #if the id for the json equals the next csv row
            csvdf.loc[csvrow, 'video_time_watched'] = jsondf.loc[jsonrow, 'Date']
            if csvrow < len(csvdf)-1:
                csvrow = csvrow+1   #go on to the next csv row
        jsonrow = jsonrow + 1  #go to the next json no matter if match or not
    return csvdf



In [64]:
#processjson and csv and insert
def mergefile(csv,json): #for Sec2gr2 files
    csvfile = pd.read_csv(f'{csv}')
    jsonfile = pd.read_json(f'{json}')
    return merged_date(processCsv(csvfile), processJson(jsonfile))

def mergefile2(csv,json): #for Sec1Gr1 files
    csvfile = pd.read_csv(f'{csv}')
    jsonfile = processJson2(f'{json}')
    return merged_date(processCsv(csvfile),jsonfile)
    
#sec1gr2_csv = pd.read_csv('pyktok_data/Sec1Gr2_26301.csv') #come back label wrong
#sec1gr2_json = pd.read_json('user_jsons/Sec2Gr2_26301.json')

In [65]:
df26301 = mergefile('pyktok_data/Sec1Gr2_26301.csv','user_jsons/Sec2Gr2_26301.json')
df33534 = mergefile('pyktok_data/Sec1Gr2_33534.csv','user_jsons/Sec2Gr2_33534.json')
df38129 = mergefile('pyktok_data/Sec1Gr2_38129.csv', 'user_jsons/Sec2Gr2_38129.json')
df10824 = mergefile2('pyktok_data/Sec1Gr1_10824.csv','user_jsons/Sec1Gr1_10824.json')
df12345 = mergefile2('pyktok_data/Sec1Gr1_12345.csv','user_jsons/Sec1Gr1_12345.json')
df50405 = mergefile2('pyktok_data/Sec1Gr1_50405.csv','user_jsons/Sec1Gr1_50405.json')

dfs = [df26301,df33534,df38129,df10824,df12345,df50405]

all_dfs = pd.concat(dfs)
all_dfs.head()

/var/folders/ls/b1_wv00s5r5fjstz103_p3rh0000gn/T/ipykernel_31834/966991795.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-03-06 19:44:41' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csvdf.loc[csvrow, 'video_time_watched'] = jsondf.loc[jsonrow, 'Date']
/var/folders/ls/b1_wv00s5r5fjstz103_p3rh0000gn/T/ipykernel_31834/966991795.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-03-08 03:42:17' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csvdf.loc[csvrow, 'video_time_watched'] = jsondf.loc[jsonrow, 'Date']
/var/folders/ls/b1_wv00s5r5fjstz103_p3rh0000gn/T/ipykernel_31834/966991795.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-0

,video_id,video_timestamp,video_time_watched,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,...,video_stickers,author_username,author_name,author_followercount,author_followingcount,author_heartcount,author_videocount,author_diggcount,author_verified,Unnamed: 0
0,7315561816673750318,2023-12-22T18:16:20,2024-03-06 19:44:41,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False,NaN
1,7315561816673750318,2023-12-22T18:16:20,2024-03-06 18:00:25,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False,NaN
2,7315561816673750318,2023-12-22T18:16:20,2024-03-06 17:58:36,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False,NaN
3,7315561816673750318,2023-12-22T18:16:20,2024-03-06 17:56:52,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False,NaN
4,7315561816673750318,2023-12-22T18:16:20,2024-03-06 17:46:03,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False,NaN


In [174]:
new = processJson(sec1gr2_json)
id = 7328816594363338030
indices = new.query('video_id == 7328816594363338030').index.tolist()
print(indices)
#print(csv.duplicated(subset='video_id').sum())

[13855]
